In [7]:
import os
import requests
import PyPDF2
from bs4 import BeautifulSoup
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain import hub
from langchain_core.prompts import PromptTemplate

os.environ["GROQ_API_KEY"] = "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"


In [8]:
# --- 1. Função que o agente usará para 'extrair' informações de um link ---
def get_text_from_url(url: str) -> str:
    """
    Extrai todo o texto visível de uma URL fornecida.
    Útil para ler o conteúdo de páginas web.
    """
    try:
        response = requests.get(url, timeout=10) # Adiciona um timeout para evitar esperas infinitas
        response.raise_for_status()  # Lança um erro para códigos de status HTTP ruins (4xx ou 5xx)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove elementos de script e estilo
        for script in soup(["script", "style"]):
            script.extract()

        # Obtém o texto
        text = soup.get_text()

        # Quebra linhas em espaços e remove espaços em branco extras
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        # Limita o tamanho do texto para evitar sobrecarga do LLM
        max_length = 4000 # Limite de caracteres para o texto extraído
        if len(text) > max_length:
            print(f"ATENÇÃO: Texto da URL truncado de {len(text)} para {max_length} caracteres.")
            text = text[:max_length] + "..." # Adiciona reticências para indicar truncamento

        return text
    except requests.exceptions.RequestException as e:
        return f"Erro ao acessar a URL: {e}"
    except Exception as e:
        return f"Erro ao processar o conteúdo da URL: {e}"

# Cria a ferramenta para extrair texto de URL
url_text_extractor_tool = Tool(
    name="get_text_from_url",
    func=get_text_from_url,
    description="Útil para extrair o conteúdo de texto de uma página web a partir de uma URL. Forneça uma URL completa como entrada."
)

In [9]:
# Inicialização do Modelo de Linguagem (LLM) ---
llm = ChatGroq(model = "llama-3.3-70b-versatile")

tools = [
    url_text_extractor_tool
    ]

In [ ]:
custom_prompt_template = """
Você é um assistente de IA útil e inteligente.
Seu nome é Sherlock Holmes.
Sua principal função é responder às perguntas do usuário.
PRIMEIRO, tente responder diretamente usando seu próprio conhecimento.
SEGUNDO, se você não souber a resposta, ou se a pergunta exigir informações muito específicas, atualizadas, ou a análise de um documento/URL, então use as ferramentas disponíveis.
Use as ferramentas APENAS quando for estritamente necessário para obter informações que você não possui.

Aqui estão as ferramentas que você pode usar:
{tools}

Use o seguinte formato:

Question: a pergunta de entrada que você deve responder
Thought: você deve sempre pensar sobre o que fazer.
Action: a ação a ser tomada, deve ser uma das [{tool_names}]
Action Input: a entrada para a ação (NÃO use aspas duplas aqui)
Observation: o resultado da ação
... (este Thought/Action/Action Input/Observation pode se repetir N vezes)
Thought: Eu sei a resposta final
Final Answer: a resposta final à pergunta original

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

# Crie um novo PromptTemplate com suas instruções personalizadas
# Certifique-se de que as variáveis de entrada (input_variables) correspondem ao que o create_react_agent espera.
prompt = PromptTemplate.from_template(custom_prompt_template)
prompt = prompt.partial(
    tools=str(tools), # Ferramentas precisam ser passadas como string para o template
    tool_names=", ".join([tool.name for tool in tools])
)

# Cria o agente
agent = create_react_agent(llm, tools, prompt)

# Cria o executor do agente
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# --- 5. Execução do Agente ---

while True:
    try:
        user_input = input("\nSua pergunta (ou 'sair' para terminar): ")
        if user_input.lower() == 'sair':
            break

        print(f"\nVocê: {user_input}")
        response = agent_executor.invoke({"input": user_input})
        print("\n--- Resposta do Agente ---")
        print('Holmes:', response["output"])
    except Exception as e:
        print(f"\nOcorreu um erro: {e}")
        print("Tente novamente ou verifique se sua chave da OpenAI está configurada corretamente.")

print("\nAgente encerrado.")



Você: Qual o seu nome?


> Entering new AgentExecutor chain...
Meu nome é Sherlock Holmes, e estou aqui para ajudar com suas perguntas de forma eficiente e inteligente.
Eu sei a resposta final
Final Answer: Sherlock Holmes

> Finished chain.

--- Resposta do Agente ---
Mr. Holmes: Sherlock Holmes

Agente encerrado.
